In [1]:
from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import stop
from phdconf import config 

import os

In [2]:
queries = load_queries(config.AUS_TOPIC_PATH)

In [3]:
# make crim queries different colour in plots 
s = {43, 44, 45, 46, 114}
for k, v in queries.items():
    v['type'] = 'specific' if k in s else 'broad'
#     if k == 44 or k == 93:
#         v['type'] = 'general'
    queries[k] = v

In [4]:
BASE_DIR = os.path.join(os.environ["HOME"], 'phd-generated')

index_names = ['filtered-phrasestop']#, config.SIGIR_INDEX_NAME]
qrel_paths = [config.AUS_QREL_PATH]#, config.SIGIR_QREL_PATH]
rel_levels = [config.AUS_REL_LEVEL]#, config.SIGIR_REL_LEVEL]
display_names = ['aus']#, 'SIGIR']

mu = 1050

base_df = load_1d_dfs(index_names, qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, mu, mu, 1)[0][0]
base_query_df = load_1d_dfs(index_names, qrel_paths, os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior'), 'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, mu, mu, 1, per_query=True)[0][0]

In [5]:
runs = ['case-topics-crim-cases-{0}.run'.format(x) for x in ['in', 'out', 'all']]
run_dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, os.path.join(BASE_DIR, "boosted"), runs, per_query=True)

In [6]:
# excluding crim cases 
# plot_diff(run_dfs[1], base_query_df[0][0], config.METRIC_NAMES, queries)

In [7]:
def get_diff(a, b, metric):
    diff = a-b
    print(diff[diff[metric] != 0.0][metric].sort_values())

# get_diff(run_dfs[1], base_query_df[0][0], 'rbp@0.80')

In [8]:
def get_num_diff(a, b, metrics):
    diff = a-b
    diff = diff.astype(bool).sum(axis=0)/len(diff)
    print(diff[diff.index.isin(metrics)].rename(metrics).round(4).to_latex())
    
# get_num_diff(run_dfs[0], base_query_df[0][0], config.METRIC_NAMES)

In [9]:
import copy

metrics = copy.copy(config.METRIC_NAMES)
del metrics['unjudged@20']

exclude_crim = [43, 44, 45, 46, 59, 64, 71, 104, 114, 117]
exclude_costs = [49, 64, 77, 78, 85, 86, 87, 116]

for r, ex in zip(['crim', 'costs'], [exclude_crim, exclude_costs]):
    print(r, ex)
    runs = ['case-topics-{0}-cases-{1}.run'.format(r, x) for x in ['in', 'out', 'all']]
    run_dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, os.path.join(BASE_DIR, "boosted"), runs, per_query=True)
    b = base_query_df.iloc[~(base_query_df.index.isin(ex))]
    a = run_dfs[1].iloc[~(run_dfs[1].index.isin(ex))]
    diff = a-b
    for m in metrics:
        print(stats.ttest_rel(a[m].values, b[m].values).pvalue)
    print(diff[metrics].rename(metrics, axis='columns').mean())
    fig = diff[metrics].rename(metrics, axis='columns').plot.box(fontsize=15, boxprops=dict(linestyle='-', linewidth=2), medianprops=dict(linestyle='-', linewidth=2), color=dict(boxes='black', whiskers='black', medians='b', caps='r'), figsize=(16, 4)).axhline(y=0, xmin=0.0, xmax=1.0, linestyle='--', linewidth=1.0, color='grey')
    fig.get_figure().savefig('figures/ausnl-{0}-filtered.pdf'.format(r))

crim [43, 44, 45, 46, 59, 64, 71, 104, 114, 117]
0.3087718754021768
nan
0.173841320647987
0.32018252154415255
0.5659925109462917
0.6300607215477304
RR        0.001004
ERR@20   -0.002994
R@20     -0.002312
R@100    -0.001471
NDCG     -0.001364
RBP      -0.000732
dtype: float64
costs [49, 64, 77, 78, 85, 86, 87, 116]


/home/danlocke/.local/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


0.24044399341519432
nan
0.0523150685685479
0.0047109546366402005
0.04684009696383688
0.5913769632150621
RR        0.003544
ERR@20   -0.003994
R@20     -0.014380
R@100    -0.031253
NDCG     -0.009948
RBP       0.001500
dtype: float64


/home/danlocke/.local/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


<Figure size 1152x288 with 1 Axes>

<Figure size 1152x288 with 1 Axes>

In [10]:
for m in metrics:
    print(b[b[m].isnull()][m])

Series([], Name: recip_rank, dtype: float64)
11   NaN
Name: err@20, dtype: float64
Series([], Name: recall_20, dtype: float64)
Series([], Name: recall_100, dtype: float64)
Series([], Name: ndcg, dtype: float64)
Series([], Name: rbp@0.80, dtype: float64)


In [11]:
with open('/home/danlocke/go/src/github.com/dan-locke/phd-data/aus/comb-aus.txt') as f:
    for line in f:
        parts = line.split()
        if (parts[0] == '11' or parts[0] == '74') and parts[3] != "0":
            print(line.strip())

74 0 2016QSC92 3


In [12]:
def get_diff_av_exclude(a, b, metrics, exclude):
    diff = a-b
    diff = diff[metrics]
    diff = diff.iloc[~(diff.index.isin(exclude))]
    diff = diff.mean()
    for m in metrics:
        print(diff[m])
        p = stats.ttest_rel(a[m], b[m]).pvalue
        if p < 0.01:
            diff[m] = "{:.4f}".format(diff[m]) + "$^{**}$"
        elif p < 0.05:
            diff[m] = "{:.4f}".format(diff[m]) + "$^{*}$"
        else:
            diff[m] = "{:.4f}".format(diff[m])
    
    return diff.rename(metrics)
    
# excluded_diff = get_diff_av_exclude(run_dfs[1], base_query_df, config.METRIC_NAMES, exclude_crim)

metrics = copy.copy(config.METRIC_NAMES)
del metrics['unjudged@20']
ex_df = pd.DataFrame()
for r, ex in zip(['crim', 'costs'], [exclude_crim, exclude_costs]):
    runs = ['case-topics-{0}-cases-{1}.run'.format(r, x) for x in ['out']]
    run_dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, os.path.join(BASE_DIR, "boosted"), runs, per_query=True)
    diff = get_diff_av_exclude(run_dfs[0], base_query_df, metrics, ex)
    ex_df[r] = diff

0.0010035294117647056
-0.0029941666666666667
-0.002311764705882353
-0.0014705882352941176
-0.0013635294117647044
-0.0007317647058823536
0.0035436781609195396
-0.003993837209302326
-0.014380459770114944
-0.03125287356321839
-0.009948275862068964
0.0014999999999999961


In [13]:
# print(excluded_diff.round(4).to_latex())
write_table('tables/filter-type', bold_max(ex_df).T.to_latex(escape=False))

In [14]:
def read_crim_file(path: str):
    lookup = set()
    with open(path) as f:
        for line in f:
            parts = line.split()
            if parts[1] == '1':
                lookup.add(parts[0])
    return lookup

def count_crim_in_res_file(path: str, crim_lookup):
    qry = {}
    with open(path) as f:
        for line in f: 
            parts = line.split()
            v = qry.get(parts[0], 0)
            if parts[2] in crim_lookup: 
                v += 1
            qry[parts[0]] = v
                           
    return qry

crim_lookup = read_crim_file('/home/danlocke/go/src/crim-feature-file/crim-cases.txt')
costs_lookup = read_crim_file('/home/danlocke/go/src/crim-feature-file/costs-cases.txt')
crim_counts = count_crim_in_res_file('/home/danlocke/phd-generated/dirichlet_prior/case-topics-filtered-stop-unigram_dir_mu_1050.00.run', crim_lookup)

In [15]:
def find_lookup_rel(path: str, lookup):
    potential = {}
    with open(path) as f:
        for line in f:
            parts = line.split()
            if parts[2] in lookup and parts[3] != '0':
                if parts[0] not in potential: 
                    potential[parts[0]] = [[], []]
                potential[parts[0]][0].append(parts[2])
                potential[parts[0]][1].append(parts[3])
    return potential
                
rel_crim = find_lookup_rel(config.AUS_QREL_PATH, crim_lookup)
rel_costs = find_lookup_rel(config.AUS_QREL_PATH, costs_lookup)

In [16]:
for q, rel_docs in rel_crim.items(): 
    print(q, queries.get(int(q)), len(rel_docs[0]))


42 None 9
43 {'topic': 'equitable jurisidiction to set aside a perfected judgment in criminal case on the basis of fraud', 'type': 'specific', 'legislation_ref': None, 'focus': 'law'} 1
44 {'topic': 'Admission of fresh or new evidence', 'type': 'specific', 'legislation_ref': None, 'focus': 'law'} 16
45 {'topic': 'Obligation of prosecution to investigate as distinct from the obligation to disclose', 'type': 'specific', 'legislation_ref': None, 'focus': 'law'} 1
46 {'topic': 'Whether incompetence of trial counsel is a basis for a criminal appeal', 'type': 'specific', 'legislation_ref': None, 'focus': 'law'} 23
47 {'topic': 'Relevant considerations for an application for summary judgment in the Federal Court', 'type': 'broad', 'legislation_ref': None, 'focus': 'generic'} 1
59 {'topic': 'motor vehicle accidents involving caravans', 'type': 'broad', 'legislation_ref': None, 'focus': 'fact'} 1
64 {'topic': "Non party costs against a party's solicitor", 'type': 'broad', 'legislation_ref': Non

In [17]:

for q, rel_docs in rel_costs.items(): 
    print(q, queries.get(int(q)), len(rel_docs[0]))

7 {'topic': 'Maintenance and champerty', 'type': 'broad', 'legislation_ref': None, 'focus': 'law'} 3
16 None 1
49 {'topic': 'Recovery of costs in estate litigation matters', 'type': 'broad', 'legislation_ref': None, 'focus': 'generic'} 9
53 {'topic': 'Causation and evidentiary standards', 'type': 'broad', 'legislation_ref': None, 'focus': 'generic'} 2
61 {'topic': 'amending pleadings after trial date set down', 'type': 'broad', 'legislation_ref': None, 'focus': 'generic'} 3
64 {'topic': "Non party costs against a party's solicitor", 'type': 'broad', 'legislation_ref': None, 'focus': 'generic'} 14
77 {'topic': 'security for costs and delay', 'type': 'broad', 'legislation_ref': None, 'focus': 'law'} 18
78 {'topic': 'success in seeking security for costs where there has been a delay', 'type': 'broad', 'legislation_ref': None, 'focus': 'fact'} 7
85 {'topic': 'Discretion as to costs', 'type': 'broad', 'legislation_ref': None, 'focus': 'law'} 37
86 {'topic': 'where a document specifies the c

In [18]:
def create_crim_count_plot(counts):
    fig = plt.figure() 
    ax = fig.add_subplot(111)
    fig.set_size_inches(16, 10)
    ticks = [str(x[0]) for x in sorted(crim_counts.items(), key=lambda x: x[1], reverse=True)]
    count_vals = sorted(crim_counts.values(), reverse=True)
    x = [i for i in range(len(counts))]
    ax.bar(x, count_vals)
    ax.set_xticks(np.arange(len(x)))
    ax.set_xticklabels(ticks, rotation=-90)
    return fig

count_plt = create_crim_count_plot(crim_counts)
count_plt.savefig('figures/crim-counts.pdf')

<Figure size 1152x720 with 1 Axes>